code that strana used to get his high incidence rate

In [ ]:
def evaluate_agent_incidence(cfg, actor_path, threshold, num_samples=100, logger=None):
    # Load agent and policy weights
    agent = PPOAgent(cfg, logger)
    agent.policy_net.load_state_dict(torch.load(actor_path, map_location="cpu"))
    agent.policy_net.eval()

    count_above = 0

    rewards = []
    distributions = []
    states = []
    state = env.reset()
    for _ in range(agent.cfg.training.max_steps_per_episode):
        mean, std = agent.policy_net(state)
        dist = torch.distributions.Normal(mean, std)
        action = dist.rsample()

        next_state, sample_reward, _ = env.step(action)

        rewards.append(sample_reward)
        distributions.append(dist)
        states.append(state)

        state = next_state

    max_idx = np.argmax(rewards)
    state = states[max_idx]
    dist = distributions[max_idx]

    for i in range(num_samples):
        env.state = state
        action = dist.rsample()
        next_state, reward, _ = env.step(action)
        if reward > threshold:
            count_above += 1
        print(f"Sample {i+1}/{num_samples}: Reward = {reward:.4f}")

    incidence_rate = count_above / num_samples
    print(f"Incidence rate (reward > {threshold}): {incidence_rate:.4f} ({count_above}/{num_samples})")
    return incidence_rate, reward